In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, HuberRegressor
import pickle
import itertools
import time
import ot
import os
from scipy.stats import rankdata
import seaborn as sns
from datetime import datetime
from scipy.signal import savgol_filter
from scipy.ndimage import gaussian_filter1d

In [2]:
df = pd.read_csv('./UniversityOfCalifornia/2013_UniversityOfCalifornia.csv')

In [ ]:
df['Position'].unique()

In [ ]:
UCB = df[(df['EmployerName'] == 'University of California, Berkeley') & (df['Position'] == 'Postdoc-Employee')]['TotalWages'].values

In [ ]:
UCLA = df[(df['EmployerName'] == 'University of California, Los Angeles') & (df['Position'] == 'Postdoc-Employee')]['TotalWages'].values

In [ ]:
bins = np.linspace(0, 200000, 100)
plt.hist(UCLA, bins, alpha=0.5, label='UCLA')
plt.hist(UCB, bins, alpha=0.5, label='UCB')
plt.legend(loc='best')
plt.show()

In [ ]:
np.median(UCB), np.median(UCLA)

In [10]:
positions = df['Position'].unique()

In [ ]:
positions

In [ ]:
for title in positions:
    if 'Instr-Grad' in title:
        print(title)

In [11]:
prof = []
for title in positions:
    if 'Teach' in title:
        print(title)
        prof.append(title)

Child Dev Ctr Teacher 1
Teacher-Lhs-Continuing
Teacher-Spec Prog
Teacher-Spec Prog-Continuing
Teacher-Unex
Teacher-Unex-Contract Yr
Teachg Asst-Gship
Teachg Asst-Non GSHIP
Supv Teacher Ed-Ay
Supv Teacher Ed-Ay-Continuing
Teacher Asst-GSHIP
Teacher Asst-Non GSHIP
Asst Teacher-Unex
Child Dev Ctr Teacher 2
Child Dev Ctr Teacher 2 Supv
Demo Teacher
Demo Teacher-Continuing
Substitute Teacher
Teachg Fellow-GSHIP
Teaching Clin Dental Ast Supv
Teachg Assistant-Non GSHIP
Teachg Fellow-Non GSHIP
Child Dev Ctr Teacher 1 Supv
Child Life Teacher 1
Child Life Teacher 1 Pd
Child Life Teacher 2
Child Life Teacher Supv
Teachg Asst-GSHIP/Non Rep
Teachg Asst-Non GSHIP/Non Rep
Assistant Teacher-Unex
Supervisor Teacher Ed-Ay
Teaching Assistant-Non GSHIP
Teaching Asst-GSHIP
Teaching Fellow-GSHIP


# 2013 Dataset has different job title abbreviation. We unify them with other dataset as follows

In [ ]:
'Assistant Prof-Ay-B/E/E', 'Associate Prof-Ay-B/E/E', 'Teaching Assistant-GSHIP'

In [ ]:
'Assoc Prof-Ay-B/E/E', 'Asst Prof-Ay-B/E/E', 'Postdoc-Employee', 'Teachg Asst-Gship'

In [4]:
n_rows = df.shape[0]

In [5]:
for idx in range(n_rows):
    if df.loc[idx, 'Position'] == 'Assistant Prof-Ay-B/E/E':
        df.loc[idx, 'Position'] = 'Asst Prof-Ay-B/E/E'
        print('Done')

In [6]:
for idx in range(n_rows):
    if df.loc[idx, 'Position'] == 'Associate Prof-Ay-B/E/E':
        df.loc[idx, 'Position'] = 'Assoc Prof-Ay-B/E/E'

In [7]:
for idx in range(n_rows):
    if df.loc[idx, 'Position'] == 'Teaching Assistant-GSHIP' or df.loc[idx, 'Position'] == 'Teachg Asst-GSHIP':
        df.loc[idx, 'Position'] = 'Teachg Asst-Gship'

In [8]:
df.to_csv('2013_UniversityOfCalifornia.csv', index=False)

# Title selection

In [ ]:
prof = ['Prof-Ay-B/E/E']

# prof = ['Prof-Ay-1/9', 'Prof-Ay-1/9-B/E/E', 'Prof-Hcomp', 'Prof Of Clin-Fy', 'Prof Of Clin-Hcomp', 
#         'Prof-Ay', 'Prof-Ay-B/E/E', 'Prof-Ay-Law', 'Prof-Fy', 'Prof-Fy-B/E/E']


# Special
# prof = ['Act Prof-Ay', 'Prof-Ay-1/9', 'Prof-Ay-1/9-B/E/E', 'Prof In Res-Ay-1/9', 'Adj Prof-Ay-1/9', 'Adj Prof-Ay-1/9-B/E/E', 
#         'VIS Prof', 'Prof In Res-Fy', 'Prof In Res-Hcomp', 'Prof In Res-Sft-Vm', 'Prof Of Clin-Sft-Vm', 'Prof-Hcomp',
#         'Prof-Sft-Vm', 'Vis Prof', 'Adj Prof-Hcomp', 'Adj Prof-Sft-Vm', 'Prof Emeritus(Wos)', 'Prof In Res-Ay', 
#         'Prof In Res-Ay-B/E/E', 'Prof Of Clin-Fy', 'Prof Of Clin-Hcomp', 'Prof-Ay', 'Prof-Ay-B/E/E', 'Prof-Ay-Law', 'Prof-Fy',
#         'Prof-Fy-B/E/E', 'Adj Prof-Ay', 'Adj Prof-Ay-1/10-Bee', 'Adj Prof-Ay-B/E/E', 'Adj Prof-Fy', 'Adj Prof-Fy-B/E/E',
#         'Act Prof-Ay-Law']

In [ ]:
asst_prof = ['Asst Prof-Ay-B/E/E']

# asst_prof = ['Asst Prof-Ay', 'Asst Prof-Ay-B/E/E', 'Asst Prof-Fy',
#              'Asst Prof Of Clin-Hcomp', 'Asst Prof-Ay-1/9', 'Asst Prof-Hcomp', 
#              'Asst Prof Of Clin-Fy']

# asst_prof = ['Asst Prof In Res-Ay', 'Asst Prof In Res-Ay-B/E/E', 'Asst Prof-Ay', 'Asst Prof-Ay-B/E/E', 'Asst Prof-Fy',
#              'Vis Asst Prof', 'Act Asst Prof-Ay', 'Asst Prof In Res-Fy-B/E/E', 'Asst Prof In Res-Hcomp',
#              'Asst Prof Of Clin-Hcomp', 'Asst Prof Of Clin-Sft-Vm', 'Asst Prof-Ay-1/9', 'Asst Prof-Hcomp', 
#              'Asst Prof-Sft-Vm', 'VIS Asst Prof', 'VIS Asst Prof-HComp', 'Asst Prof Of Clin-Fy', 'Asst Prof In Res-Fy',
#              'Vis Asst Prof-Hcomp', 'Act Asst Prof-Ay-B/E/E']

In [ ]:
assoc_prof = ['Assoc Prof-Ay-B/E/E']

# assoc_prof = ['Assoc Prof Of Clin-Hcomp', 'Assoc Prof-Ay', 'Assoc Prof-Ay-B/E/E', 'Assoc Prof-Fy',
#               'Assoc Prof-Fy-B/E/E', 'Assoc Prof-Hcomp', 'Assoc Prof Of Clin-Fy']

# assoc_prof = ['Assoc Prof In Res-Ay', 'Assoc Prof Of Clin-Hcomp', 'Assoc Prof-Ay', 'Assoc Prof-Ay-B/E/E', 'Assoc Prof-Fy',
#               'Vis Assoc Prof', 'Act Assoc Prof-Ay', 'Assoc Prof In Res-Fy', 'Assoc Prof In Res-Hcomp', 
#               'Assoc Prof In Res-Sft-Vm', 'Assoc Prof Of Clin-Sft-Vm', 'Assoc Prof-Fy-B/E/E', 'Assoc Prof-Hcomp',
#               'Assoc Prof-Sft-Vm', 'VIS Assoc Prof', 'VIS Assoc Prof-HComp', 'Act Assoc Prof-Ay-B/E/E', 
#               'Assoc Prof Of Clin-Fy', 'Assoc Prof In Res-Ay-B/E/E']

In [ ]:
graduates = ['Gsr-Full Fee Rem', 'Gsr-No Rem', 'Gsr-Partial Fee Rem', 'Gsr-Tuit & Fee Rem', 
             'Gsr-Full Tuit&Partial Fee Rem', 'Teachg Asst-Gship']

# graduates = ['Gsr-Full Fee Rem', 'Gsr-No Rem', 'Gsr-Partial Fee Rem', 'Gsr-Tuit & Fee Rem', 
#              'Gsr-Full Tuit&Partial Fee Rem', 'Gsr-Tuit & Fee Rem-UCSD-Grp B', 'Gsr-Tuit & Fee Rem-UCSD-Grp C',
#              'Gsr-Tuit & Fee Rem-UCSD-Grp D', 'Gsr-Tuit & Fee Rem-UCSD-Grp E', 'Gsr-Tuit & Fee Rem-UCSD-Grp F',
#              'Gsr-Tuit & Fee Rem-UCSD-Grp G']

# graduates = ['Gsr-Full Fee Rem', 'Gsr-No Rem', 'Gsr-Partial Fee Rem', 'Gsr-Tuit & Fee Rem', 
#              'Gsr-Full Tuit&Partial Fee Rem', 'Gsr-Tuit & Fee Rem-UCSD-Grp B', 'Gsr-Tuit & Fee Rem-UCSD-Grp C',
#              'Gsr-Tuit & Fee Rem-UCSD-Grp D', 'Gsr-Tuit & Fee Rem-UCSD-Grp E', 'Gsr-Tuit & Fee Rem-UCSD-Grp F',
#              'Gsr-Tuit & Fee Rem-UCSD-Grp G', 'Teachg Asst-1/10-Gship',
#              'Teachg Asst-1/10-Non Gship', 'Teachg Asst-Gship', 'Teachg Asst-Non Gship',
#              'Teachg Asst-Gship/Non Rep', 'Teachg Asst-Non Gship/Non Rep']

In [ ]:
postdoc = ['Postdoc-Employee']
# postdoc = ['Intrm Postdoc Scholar-Employee', 'Postdoc-Employee', 'Postdoc-Employee Nex', 'Postdoc-Fellow',
#            'Postdoc-Paid Direct']
# Oth Post-Md Train 2-8/Non Rep
# Pgy1 Post Pharmd Trainee
# Pgy2 Spec Post Pharmd Trainee
# Oth Post Dds/Non Rep
# Oth Post Dds/Rep
# Post Dds I-Vi/Non Rep
# Post Dds I-Vi/Rep
# Oth Post-Md Train 2-8/Rep
# Stipend-Oth Post-Md Train

In [ ]:
head_count = df.groupby(['Position'])['TotalWages'].count().reset_index()

In [ ]:
head_count.sort_values(by=['TotalWages'], ascending=False).head(10)

In [ ]:
head_count[head_count['Position'].isin(prof + assoc_prof + asst_prof + postdoc + graduates)]

In [ ]:
PROF = df[df['Position'].isin(prof)]
PROF_salary = PROF.groupby(['EmployerName'])['TotalWages'].max().reset_index()
PROF_salary.sort_values(by=['TotalWages'])

In [ ]:
df[(df['Position'].isin(prof)) & (df['EmployerName'] == 'University of California, San Francisco')][['Position']]

In [ ]:
ASOC_PROF = df[df['Position'].isin(assoc_prof)]
ASOC_PROF_salary = ASOC_PROF.groupby(['EmployerName'])['TotalWages'].median().reset_index()
ASOC_PROF_salary.sort_values(by=['TotalWages'])

In [ ]:
AP = df[df['Position'].isin(asst_prof)]
AP_salary = AP.groupby(['EmployerName'])['TotalWages'].median().reset_index()
AP_salary.sort_values(by=['TotalWages'])

In [ ]:
POST = df[df['Position'].isin(postdoc)] # Postdoc-Fellow
POST_salary = POST.groupby(['EmployerName'])['TotalWages'].median().reset_index()
POST_salary.sort_values(by=['TotalWages'])

In [ ]:
POST = df[df['Position'] == 'Teachg Asst-Gship']
POST_salary = POST.groupby(['EmployerName'])['TotalWages'].median().reset_index()
POST_salary.sort_values(by=['TotalWages'])

In [ ]:
POST[['Year', 'EmployerName', 'Position', 'TotalWages']]

In [ ]:
POST[['TotalWages']].min()

In [ ]:
plt.hist(POST[POST['EmployerName'] == 'University of California, Berkeley']['TotalWages'])

In [ ]:
POST.groupby(['EmployerName'])['TotalWages'].count()

In [ ]:
GRAD = df[df['Position'].isin(graduates)]
GRAD_salary = GRAD.groupby(['EmployerName'])['TotalWages'].mean().reset_index()
GRAD_salary.sort_values(by=['TotalWages'])

In [ ]:
GRAD.groupby(['EmployerName'])['TotalWages'].count()